In [18]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
# Path to chromedriver
!pip install chromedriver

In [63]:
import pandas as pd
import pymongo
import requests
from splinter import Browser

In [20]:
# Set the executable path and initialize the chrome browser in splinter
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [C:\Users\rashe\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


## Visit the NASA mars news site

In [21]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [23]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
sitesoup = BeautifulSoup(html, 'html.parser')

siteelement = sitesoup.select_one('ul.item_list li.slide')

In [24]:
# .find() the content title and save it as `news_title`
siteelement.find("div", class_='content_title')

<div class="content_title"><a href="/news/8963/nasas-perseverance-rover-begins-its-first-science-campaign-on-mars/" target="_self">NASA's Perseverance Rover Begins Its First Science Campaign on Mars</a></div>

In [26]:
# .find() the paragraph text i.e to filter out the news title without the a href tags
newstitle = siteelement.find("div", class_='content_title').get_text()
newstitle

"NASA's Perseverance Rover Begins Its First Science Campaign on Mars"

In [28]:
# Use the parent element to find the paragraph text i.e to find Main header news that housed the news title
parentnews = siteelement.find('div', class_="article_teaser_body").get_text()
parentnews

'The six-wheeled scientist is heading south to explore Jezero Crater’s lakebed in search of signs of ancient microbial life.'

## JPL Space Images Featured Image

In [60]:
# Visit JPL space images Mars URL 
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [64]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [66]:
# find the relative image url
img = soup.findAll('img', class_= "headerimage fade-in")
imgfound= img[0].attrs['src']
imgfound

'image/featured/mars3.jpg'

In [70]:
# Use the base url to create an absolute url
imgfound_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'+imgfound
imgfound_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

## Mars Facts

In [71]:
# Create a dataframe from the space-facts.com mars page

goin2mars = 'https://space-facts.com/mars/'
browser.visit(goin2mars)

In [78]:
# clean the dataframe and export to HTML
mars_data = pd.read_html(goin2mars)[1]
mars_data

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-87 to -5 °C,-88 to 58°C


In [79]:
cleaned_mars_data = mars_data.rename(columns= {'Mars - Earth Comparison': 'Attributes'}).drop(columns = ["Earth"])
cleaned_mars_data 

,Attributes,Mars
0,Diameter:,"6,779 km"
1,Mass:,6.39 × 10^23 kg
2,Moons:,2
3,Distance from Sun:,"227,943,824 km"
4,Length of Year:,687 Earth days
5,Temperature:,-87 to -5 °C


In [80]:
cleaned_mars_data.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Attributes</th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Moons:</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n  </tbody>\n</table>'

## Hemispheres

In [81]:
# visit the USGS astrogeology page for hemisphere data from Mars
url ="https://astrogeology.usgs.gov"
hemisphereUrl = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphereUrl)

In [82]:
# Convert the browser html to a soup object and prettify
html_hemisphere = browser.html
soup_hemisphere = BeautifulSoup(html_hemisphere,'html.parser')

In [ ]:
# Finds images in divs with class=item
imgsfound = soup_hemisphere.findAll('div', class_='item')

In [88]:
# First, get a list of all of the hemispheres
# Next, loop through those links, click the link, find the sample anchor, return the href
# We have to find the elements on each loop to avoid a stale element exception
# Next, we find the Sample image anchor tag and extract the href
# Get Hemisphere title
# Append hemisphere object to list
    
list_imgs_url = []
imgtitles = soup_hemisphere.findAll('h3')

for title in range(len(imgtitles)):
    hemisphere ={}
    imgtitles = soup_hemisphere.findAll('h3')[title].text
    
    imgUrl = url + soup_hemisphere.findAll('a', class_='itemLink product-item')[title]['href']
    browser.visit(imgUrl)
    
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    imagelinktitle = soup.find('div', class_='downloads').find('a')['href']
    
    hemisphere["title"] = imgtitles
    hemisphere["imgUrl"] = imagelinktitle
    
    list_imgs_url.append(hemisphere)
    

In [89]:
# view the hemisphere urls to make sure they look good
list_imgs_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'imgUrl': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'imgUrl': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'imgUrl': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'imgUrl': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}]

In [90]:
 # Finally, we navigate backwards with browser.back()
browser.quit()